In [ ]:
import json
import os
import StringIO

import cellprofiler.image
import cellprofiler.measurement
import cellprofiler.object
import cellprofiler.pipeline
import cellprofiler.preferences
import cellprofiler.workspace
import imagej
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import skimage.io

In [ ]:
def run_pipeline(pipeline_filename, image_dict):
    cellprofiler.preferences.set_headless()
    
    # Create and load the pipeline
    pipeline = cellprofiler.pipeline.Pipeline()
    pipeline.load(pipeline_filename)
    
    # Create the image set, and add the image data
    image_set_list = cellprofiler.image.ImageSetList()
    image_set = image_set_list.get_image_set(0)
    for image_name, input_pixels in image_dict.iteritems():
        image_set.add(image_name, cellprofiler.image.Image(input_pixels))
        
    # Persist the object set here (for now, see workspace TODO)
    object_set = cellprofiler.object.ObjectSet()

    # We can only run one group -- set the group index to 1.
    measurements = cellprofiler.measurement.Measurements()
    measurements.group_index = 1

    # Run the modules!
    for module in pipeline.modules():
        # Yes, we really do have to create a new workspace for each module
        # because the module attribute is required. Go team.
        workspace = cellprofiler.workspace.Workspace(
            image_set=image_set,
            image_set_list=image_set_list,
            measurements=measurements, 
            module=module,
            object_set=object_set, 
            pipeline=pipeline
        )
        
        module.prepare_run(workspace)
        module.run(workspace)
        module.post_run(workspace)
    
    # The workspace object has access to the measurements
    # and the image set/image set list which can be used
    # to use/view/store/whatever output data.
    return workspace


def objects2df(measurements, objects_name):
    features = measurements.get_feature_names(objects_name)
    
    n_features = len(features)
    n_objects = int(measurements.get_measurement("Image", "Count_{}".format(objects_name)))
    
    data = np.empty((n_objects, n_features))
    
    for feature_idx, feature in enumerate(features):
        data[:, feature_idx] = measurements.get_measurement(objects_name, feature)
    
    return pd.DataFrame(
        data=data,
        index=np.arange(1, n_objects + 1),
        columns=features
    )


def data2image(data):
    return Image.open(StringIO.StringIO(data))


def show(image):
    plt.imshow(image, interpolation='nearest', cmap='gray')
    plt.show()

In [ ]:
ij = imagej.IJ()

img_id = ij.upload('data/images/1-162hrh2ax2.tif')

orig_png = ij.retrieve(img_id, format='png')
orig_pil = data2image(orig_png)
orig_np = np.array(orig_pil).copy()

script_eval = ij.find('ScriptEval')[0]

script = """
#@ CommandService commandService
#@ DisplayService displayService
#@both Img image
from net.imagej.plugins.commands.neigh import FindEdges
imageDisplay = displayService.createDisplay(image)
commandService.run(FindEdges, True, {'display': imageDisplay})
"""

_ = ij.run(script_eval, {'language': 'python', 'script': script, 'args': {'image': img_id}})

enhanced_png = ij.retrieve(img_id, format='png')
enhanced_pil = data2image(enhanced_png)
enhanced_np = np.array(enhanced_pil)

In [ ]:
images = {
    'EnhancedGreen': enhanced_np,
    'OrigBlue': skimage.io.imread('data/images/1-162hrhoe2.tif'),
    'OrigGreen': orig_np
}

pipeline_filename = 'data/ExampleSpeckles.cppipe'

workspace = run_pipeline(pipeline_filename, images)

In [ ]:
df = objects2df(workspace.measurements, "Nuclei")
df.head()

In [ ]:
nuclei = workspace.object_set.get_objects("Nuclei")
skimage.io.imshow(nuclei.segmented)